# Задание 1

In [113]:
import pandas as pd

In [178]:
geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

stats = pd.read_csv('keywords.csv')
stats.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [179]:
for regionName in geo_data:
    for cityName in geo_data[regionName]:
        print('{0} - {1}'.format(regionName, cityName))

Центр - москва
Центр - тула
Центр - ярославль
Северо-Запад - петербург
Северо-Запад - псков
Северо-Запад - мурманск
Дальний Восток - владивосток
Дальний Восток - сахалин
Дальний Восток - хабаровск


In [180]:

def hasSubstr(row, line):
    """Возвращает True, если строка содержится в ячейке"""
    for cell in row.tolist():
        if(str(cell).lower().find(str(line).lower()) >= 0):
            return True
    return False
def decodeRegion(row):
    """Возвращает регион определенный по строке"""
    for regionName in geo_data:
        for cityName in geo_data[regionName]:
            if(hasSubstr(row, cityName)):
                return regionName
    return 'undefined'


In [181]:
stats['region'] = stats.apply(decodeRegion, axis=1)
stats['region'].value_counts()

undefined         99253
Центр               387
Северо-Запад        266
Дальний Восток       94
Name: region, dtype: int64

# Задание 2

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
    - оценка 2 и меньше - низкий рейтинг
    - оценка 4 и меньше - средний рейтинг
    - оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [182]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [183]:
def decodeRating(row):
    """Возвращает рейтинг классификатор:
        - оценка 2 и меньше - низкий рейтинг
        - оценка 4 и меньше - средний рейтинг
        - оценка 4.5 и 5 - высокий рейтинг
    """
    if(row['rating'] <= 2.0):
        return 'низкий рейтинг'
    if(2 < row['rating'] <= 4.0):
        return 'средний рейтинг'
    return 'высокий рейтинг'


In [184]:
ratings['class'] = ratings.apply(decodeRating, axis=1)
ratings.head()

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,средний рейтинг
1,1,1029,3.0,1260759179,средний рейтинг
2,1,1061,3.0,1260759182,средний рейтинг
3,1,1129,2.0,1260759185,низкий рейтинг
4,1,1172,4.0,1260759205,средний рейтинг


In [185]:
ratings['class'].value_counts()

средний рейтинг    63801
высокий рейтинг    22818
низкий рейтинг     13385
Name: class, dtype: int64

# Задание 3

In [186]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [187]:
groups = ratings.groupby(['userId'])
groups = groups.filter(lambda x: len(x) >= 100).reset_index()
groups1 = groups.groupby(['userId']).agg({'timestamp':['min', 'max', 'count']}).reset_index()
groups1.columns = ['userId', 'tsmin', 'tsmax', 'cnt']
groups1.head()

,userId,tsmin,tsmax,cnt
0,4,949778714,949982274,204
1,5,1163373044,1163375145,100
2,8,1154389340,1154474527,116
3,15,997937239,1469330735,1700
4,17,1127468587,1127476640,363


In [188]:
groups1['lifetime'] = groups1.apply(lambda x: (x['tsmax'] - x['tsmin']), axis=1)
groups1.head()

,userId,tsmin,tsmax,cnt,lifetime
0,4,949778714,949982274,204,203560
1,5,1163373044,1163375145,100,2101
2,8,1154389340,1154474527,116,85187
3,15,997937239,1469330735,1700,471393496
4,17,1127468587,1127476640,363,8053


In [189]:
avg_lt = groups1['lifetime'].mean()/(3600*24)
print('Среднее значение Lifetime киноманов: {:.1f} дней'.format(avg_lt))

Среднее значение Lifetime киноманов: 455.2 дней


# Задание 4

In [202]:
years = [str(i) for i in range(1950,2010+1)]
ratings = pd.read_csv('ml-latest-small/ratings.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')
ratingsByMovie = ratings.groupby(['movieId'])['rating'].mean().reset_index()
ratingsByMovie.head()

,movieId,rating
0,1,3.872470
1,2,3.401869
2,3,3.161017
3,4,2.384615
4,5,3.267857


In [203]:

movieRatings = ratingsByMovie.merge(movies, on='movieId', how='left')
def production_year(row):
    """Возвращает год производства фильма"""
    for year in years:
        if year in row['title']:
            return year
    return '1900'
movieRatings['year'] = movieRatings.apply(production_year, axis=1)
movieRatings.head()

,movieId,rating,title,genres,year
0,1,3.872470,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,3.401869,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,3.161017,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,2.384615,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,3.267857,Father of the Bride Part II (1995),Comedy,1995


In [208]:
yearRatings = movieRatings.groupby(['year'])['rating'].mean().reset_index().sort_values('rating', ascending=False)
yearRatings.head()

,year,rating
2,1951,3.906653
3,1952,3.802318
13,1962,3.772962
4,1953,3.738878
8,1957,3.693322
